In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import cv2
import numpy as np

from tensorflow.keras.applications import EfficientNetB7

import tensorflow.keras
from tensorflow.keras import applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import *

In [ ]:
train_video_path = '/content/drive/MyDrive/Egy_data/batches/'
saved_nparray_path = '/content/drive/MyDrive/Egy_data/train_nparray/data/' #this path will be deleted if we have good resources
train_nparray_path = '/content/drive/MyDrive/Egy_data/train_nparray/'

img_height , img_width = 600, 600  # dimension of each frame in video
seq_len = 40 #the number of images we pass as one sequence (only 70 frames per video) -more frames/ better results/ computationally expensive

classes = [0, 1, 2, 3]

In [ ]:
def conv_base():
    feature_extractor =  tensorflow.keras.applications.EfficientNetB7(input_shape=(img_height,img_width,3),
                              include_top=False, weights='imagenet',pooling="avg")

    preprocess_input = tensorflow.keras.applications.efficientnet.preprocess_input
    inputs = tensorflow.keras.Input((img_height, img_width, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return tensorflow.keras.Model(inputs, outputs, name="conv_base")

conv_base = conv_base()
conv_base.summary()

Model: "conv_base"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 600, 600, 3)]     0         
                                                                 
 efficientnetb7 (Functional)  (None, 2560)             64097687  
                                                                 
Total params: 64,097,687
Trainable params: 63,786,960
Non-trainable params: 310,727
_________________________________________________________________


In [ ]:
part1=0
part2=40
part3=80
part4=120
part5=160
part6=200
part7=240

currentPart = -1
def frames_extraction(video_path):

    frames_list = []

    #creating an object by capturing the given video
    #This object can be used to read frames from the captured video
    vidObj = cv2.VideoCapture(video_path)

    #Counter variable - will be incrementing upto the seq_len
    seqCount = 1
    #skipCount = 2
    defect = 0
    successCount = 0
    start = 0

    #The count variable ensures that the number of extracted frames should be equal to the seq_len
    while seqCount <= seq_len:

        #Reading one frame at a time from the video, it returns status (success) and the actual frame (image)
        success, image = vidObj.read()
        #print(success)
        #If it is not able to read the frame properly then success = 0, and code will jump to else part
        #if success and skipCount == 0:
        if success and start>=currentPart:
            image = cv2.resize(image, (img_height, img_width))
            frames_list.append(image)
            seqCount += 1
            #skipCount = 2
            successCount += 1


        elif success and start<currentPart:
            start +=1
        #elif skipCount > 0:
        #  skipCount -= 1

        elif (defect + seqCount) >=300:
          break

        else:
            '''Defect frame will not be added to list of frames and if number of extracted frames is less than the seq_len, video will be rejected, in next section'''
            defect += 1
            print("Defected frame at ", defect, " ", video_path)

    print(start)
    print("Defected", defect)
    print("successCount", successCount)
    frames = np.asarray(frames_list)
    vidObj.release()
    return frames


In [ ]:
def create_Data(batch_dir):
    X = []
    Y = []

    classes_list = os.listdir(batch_dir)
    print(classes_list)

    for c in classes_list:
        print(c)
        # creating the list of videos for the current class
        files_list = os.listdir(os.path.join(batch_dir, c))
        # we’ll pass videos one by one for frames extraction
        for f in files_list:
            #extracting the frames from the given video
            video_path = (os.path.join(os.path.join(batch_dir, c), f))
            #print(video_path)
            frames = frames_extraction(video_path)
            # Check whether the number of frames is equal to the seq_len, if yes -> process, otherwise reject the current video
            if len(frames) == seq_len:
                print(np.asarray(frames).shape)
                #appending the sequence of frames to the input list X
                X.append(conv_base.predict(frames))
                #X.append(frames)
                #we are creating one hot encoding for target variable
                #create the list y of length number of classes, where each element is 0
                y = [0]*len(classes)
                #We find the index of current class c in the classes list and making corresponding element 1 in y
                y[classes.index(int(c))] = 1
                #Now we append the list y into Y
                Y.append(y)

    X = np.asarray(X)
    Y = np.asarray(Y)
    return X, Y

In [ ]:
currentPart = part1
batches_list = os.listdir(train_video_path)
print(batches_list)
saved_batches = os.listdir(saved_nparray_path) #no need for this if we have good reources
print(saved_batches)
counter = 0
for batch in batches_list:
        if batch+".npy" in saved_batches: #no need for the if we have no resources
          skippedBatch = np.load(os.path.join(train_nparray_path, "data" , batch+".npy"))
          print(batch, "skipped", skippedBatch.shape)
          continue
        print(batch)
        x_batch, y_batch = create_Data(os.path.join(train_video_path, batch))
        x_batch = np.asarray(x_batch)
        y_batch = np.asarray(y_batch)
        np.save(os.path.join(train_nparray_path, "data", batch), x_batch)
        np.save(os.path.join(train_nparray_path, "label", batch), y_batch)

        counter += 1
        print(counter)

['Batch9', 'Batch88', 'Batch87', 'Batch86', 'Batch85', 'Batch84', 'Batch83', 'Batch82', 'Batch81', 'Batch80', 'Batch8', 'Batch79', 'Batch78', 'Batch77', 'Batch76', 'Batch75', 'Batch74', 'Batch73', 'Batch72', 'Batch71', 'Batch70', 'Batch7', 'Batch69', 'Batch61', 'Batch62', 'Batch68', 'Batch66', 'Batch63', 'Batch67', 'Batch64', 'Batch65', 'Batch60', 'Batch6', 'Batch59', 'Batch58', 'Batch57', 'Batch56', 'Batch54', 'Batch51', 'Batch49', 'Batch52', 'Batch55', 'Batch53', 'Batch50', 'Batch5', 'Batch48', 'Batch47', 'Batch43', 'Batch46', 'Batch40', 'Batch42', 'Batch45', 'Batch4', 'Batch41', 'Batch44', 'Batch39', 'Batch38', 'Batch37', 'Batch36', 'Batch35', 'Batch34', 'Batch29', 'Batch27', 'Batch30', 'Batch3', 'Batch32', 'Batch28', 'Batch33', 'Batch31', 'Batch26', 'Batch25', 'Batch24', 'Batch23', 'Batch22', 'Batch21', 'Batch16', 'Batch19', 'Batch18', 'Batch20', 'Batch15', 'Batch2', 'Batch17', 'Batch14', 'Batch13', 'Batch12', 'Batch11', 'Batch10', 'Batch1']
['Batch9.npy', 'Batch88.npy', 'Batch87.n

In [ ]:
currentPart = part2
batches_list = os.listdir(train_video_path)
print(batches_list)
saved_batches = os.listdir(saved_nparray_path) #no need for this if we have good reources
print(saved_batches)
counter = 0
for batch in batches_list:
        if batch+"Dup1.npy" in saved_batches: #no need for the if we have no resources
          skippedBatch = np.load(os.path.join(train_nparray_path, "data" , batch+".npy"))
          print(batch, "Dup1 skipped", skippedBatch.shape)
          continue
        print(batch)
        x_batch, y_batch = create_Data(os.path.join(train_video_path, batch))
        x_batch = np.asarray(x_batch)
        y_batch = np.asarray(y_batch)
        np.save(os.path.join(train_nparray_path, "data", batch+"Dup1"), x_batch)
        np.save(os.path.join(train_nparray_path, "label", batch+"Dup1"), y_batch)

        counter += 1
        print(counter)

['Batch9', 'Batch88', 'Batch87', 'Batch86', 'Batch85', 'Batch84', 'Batch83', 'Batch82', 'Batch81', 'Batch80', 'Batch8', 'Batch79', 'Batch78', 'Batch77', 'Batch76', 'Batch75', 'Batch74', 'Batch73', 'Batch72', 'Batch71', 'Batch70', 'Batch7', 'Batch69', 'Batch61', 'Batch62', 'Batch68', 'Batch66', 'Batch63', 'Batch67', 'Batch64', 'Batch65', 'Batch60', 'Batch6', 'Batch59', 'Batch58', 'Batch57', 'Batch56', 'Batch54', 'Batch51', 'Batch49', 'Batch52', 'Batch55', 'Batch53', 'Batch50', 'Batch5', 'Batch48', 'Batch47', 'Batch43', 'Batch46', 'Batch40', 'Batch42', 'Batch45', 'Batch4', 'Batch41', 'Batch44', 'Batch39', 'Batch38', 'Batch37', 'Batch36', 'Batch35', 'Batch34', 'Batch29', 'Batch27', 'Batch30', 'Batch3', 'Batch32', 'Batch28', 'Batch33', 'Batch31', 'Batch26', 'Batch25', 'Batch24', 'Batch23', 'Batch22', 'Batch21', 'Batch16', 'Batch19', 'Batch18', 'Batch20', 'Batch15', 'Batch2', 'Batch17', 'Batch14', 'Batch13', 'Batch12', 'Batch11', 'Batch10', 'Batch1']
['Batch9.npy', 'Batch88.npy', 'Batch87.n

In [ ]:
currentPart = part3
batches_list = os.listdir(train_video_path)
print(batches_list)
saved_batches = os.listdir(saved_nparray_path) #no need for this if we have good reources
print(saved_batches)
counter = 0
for batch in batches_list:
        if batch+"Dup2.npy" in saved_batches: #no need for the if we have no resources
          skippedBatch = np.load(os.path.join(train_nparray_path, "data" , batch+".npy"))
          print(batch, "Dup2 skipped", skippedBatch.shape)
          continue
        print(batch)
        x_batch, y_batch = create_Data(os.path.join(train_video_path, batch))
        x_batch = np.asarray(x_batch)
        y_batch = np.asarray(y_batch)
        np.save(os.path.join(train_nparray_path, "data", batch+"Dup2"), x_batch)
        np.save(os.path.join(train_nparray_path, "label", batch+"Dup2"), y_batch)

        counter += 1
        print(counter)

['Batch9', 'Batch88', 'Batch87', 'Batch86', 'Batch85', 'Batch84', 'Batch83', 'Batch82', 'Batch81', 'Batch80', 'Batch8', 'Batch79', 'Batch78', 'Batch77', 'Batch76', 'Batch75', 'Batch74', 'Batch73', 'Batch72', 'Batch71', 'Batch70', 'Batch7', 'Batch69', 'Batch61', 'Batch62', 'Batch68', 'Batch66', 'Batch63', 'Batch67', 'Batch64', 'Batch65', 'Batch60', 'Batch6', 'Batch59', 'Batch58', 'Batch57', 'Batch56', 'Batch54', 'Batch51', 'Batch49', 'Batch52', 'Batch55', 'Batch53', 'Batch50', 'Batch5', 'Batch48', 'Batch47', 'Batch43', 'Batch46', 'Batch40', 'Batch42', 'Batch45', 'Batch4', 'Batch41', 'Batch44', 'Batch39', 'Batch38', 'Batch37', 'Batch36', 'Batch35', 'Batch34', 'Batch29', 'Batch27', 'Batch30', 'Batch3', 'Batch32', 'Batch28', 'Batch33', 'Batch31', 'Batch26', 'Batch25', 'Batch24', 'Batch23', 'Batch22', 'Batch21', 'Batch16', 'Batch19', 'Batch18', 'Batch20', 'Batch15', 'Batch2', 'Batch17', 'Batch14', 'Batch13', 'Batch12', 'Batch11', 'Batch10', 'Batch1']
['Batch9.npy', 'Batch88.npy', 'Batch87.n

In [ ]:

currentPart = part4
batches_list = os.listdir(train_video_path)
print(batches_list)
saved_batches = os.listdir(saved_nparray_path) #no need for this if we have good reources
print(saved_batches)
counter = 0
for batch in batches_list:
        if batch+"Dup3.npy" in saved_batches: #no need for the if we have no resources
          skippedBatch = np.load(os.path.join(train_nparray_path, "data" , batch+".npy"))
          print(batch, "Dup3 skipped", skippedBatch.shape)
          continue
        print(batch)
        x_batch, y_batch = create_Data(os.path.join(train_video_path, batch))
        x_batch = np.asarray(x_batch)
        y_batch = np.asarray(y_batch)
        np.save(os.path.join(train_nparray_path, "data", batch+"Dup3"), x_batch)
        np.save(os.path.join(train_nparray_path, "label", batch+"Dup3"), y_batch)

        counter += 1
        print(counter)


['Batch9', 'Batch88', 'Batch87', 'Batch86', 'Batch85', 'Batch84', 'Batch83', 'Batch82', 'Batch81', 'Batch80', 'Batch8', 'Batch79', 'Batch78', 'Batch77', 'Batch76', 'Batch75', 'Batch74', 'Batch73', 'Batch72', 'Batch71', 'Batch70', 'Batch7', 'Batch69', 'Batch61', 'Batch62', 'Batch68', 'Batch66', 'Batch63', 'Batch67', 'Batch64', 'Batch65', 'Batch60', 'Batch6', 'Batch59', 'Batch58', 'Batch57', 'Batch56', 'Batch54', 'Batch51', 'Batch49', 'Batch52', 'Batch55', 'Batch53', 'Batch50', 'Batch5', 'Batch48', 'Batch47', 'Batch43', 'Batch46', 'Batch40', 'Batch42', 'Batch45', 'Batch4', 'Batch41', 'Batch44', 'Batch39', 'Batch38', 'Batch37', 'Batch36', 'Batch35', 'Batch34', 'Batch29', 'Batch27', 'Batch30', 'Batch3', 'Batch32', 'Batch28', 'Batch33', 'Batch31', 'Batch26', 'Batch25', 'Batch24', 'Batch23', 'Batch22', 'Batch21', 'Batch16', 'Batch19', 'Batch18', 'Batch20', 'Batch15', 'Batch2', 'Batch17', 'Batch14', 'Batch13', 'Batch12', 'Batch11', 'Batch10', 'Batch1']
['Batch9.npy', 'Batch88.npy', 'Batch87.n

In [ ]:

currentPart = part5
batches_list = os.listdir(train_video_path)
print(batches_list)
saved_batches = os.listdir(saved_nparray_path) #no need for this if we have good reources
print(saved_batches)
counter = 0
for batch in batches_list:
        if batch+"Dup4.npy" in saved_batches: #no need for the if we have no resources
          skippedBatch = np.load(os.path.join(train_nparray_path, "data" , batch+".npy"))
          print(batch, "Dup4 skipped", skippedBatch.shape)
          continue
        print(batch)
        x_batch, y_batch = create_Data(os.path.join(train_video_path, batch))
        x_batch = np.asarray(x_batch)
        y_batch = np.asarray(y_batch)
        np.save(os.path.join(train_nparray_path, "data", batch+"Dup4"), x_batch)
        np.save(os.path.join(train_nparray_path, "label", batch+"Dup4"), y_batch)

        counter += 1
        print(counter)


['Batch9', 'Batch88', 'Batch87', 'Batch86', 'Batch85', 'Batch84', 'Batch83', 'Batch82', 'Batch81', 'Batch80', 'Batch8', 'Batch79', 'Batch78', 'Batch77', 'Batch76', 'Batch75', 'Batch74', 'Batch73', 'Batch72', 'Batch71', 'Batch70', 'Batch7', 'Batch69', 'Batch61', 'Batch62', 'Batch68', 'Batch66', 'Batch63', 'Batch67', 'Batch64', 'Batch65', 'Batch60', 'Batch6', 'Batch59', 'Batch58', 'Batch57', 'Batch56', 'Batch54', 'Batch51', 'Batch49', 'Batch52', 'Batch55', 'Batch53', 'Batch50', 'Batch5', 'Batch48', 'Batch47', 'Batch43', 'Batch46', 'Batch40', 'Batch42', 'Batch45', 'Batch4', 'Batch41', 'Batch44', 'Batch39', 'Batch38', 'Batch37', 'Batch36', 'Batch35', 'Batch34', 'Batch29', 'Batch27', 'Batch30', 'Batch3', 'Batch32', 'Batch28', 'Batch33', 'Batch31', 'Batch26', 'Batch25', 'Batch24', 'Batch23', 'Batch22', 'Batch21', 'Batch16', 'Batch19', 'Batch18', 'Batch20', 'Batch15', 'Batch2', 'Batch17', 'Batch14', 'Batch13', 'Batch12', 'Batch11', 'Batch10', 'Batch1']
['Batch9.npy', 'Batch88.npy', 'Batch87.n

In [ ]:

currentPart = part6
batches_list = os.listdir(train_video_path)
print(batches_list)
saved_batches = os.listdir(saved_nparray_path) #no need for this if we have good reources
print(saved_batches)
counter = 0
for batch in batches_list:
        if batch+"Dup5.npy" in saved_batches: #no need for the if we have no resources
          skippedBatch = np.load(os.path.join(train_nparray_path, "data" , batch+".npy"))
          print(batch, "Dup5 skipped", skippedBatch.shape)
          continue
        print(batch)
        x_batch, y_batch = create_Data(os.path.join(train_video_path, batch))
        x_batch = np.asarray(x_batch)
        y_batch = np.asarray(y_batch)
        np.save(os.path.join(train_nparray_path, "data", batch+"Dup5"), x_batch)
        np.save(os.path.join(train_nparray_path, "label", batch+"Dup5"), y_batch)

        counter += 1
        print(counter)


['Batch9', 'Batch88', 'Batch87', 'Batch86', 'Batch85', 'Batch84', 'Batch83', 'Batch82', 'Batch81', 'Batch80', 'Batch8', 'Batch79', 'Batch78', 'Batch77', 'Batch76', 'Batch75', 'Batch74', 'Batch73', 'Batch72', 'Batch71', 'Batch70', 'Batch7', 'Batch69', 'Batch61', 'Batch62', 'Batch68', 'Batch66', 'Batch63', 'Batch67', 'Batch64', 'Batch65', 'Batch60', 'Batch6', 'Batch59', 'Batch58', 'Batch57', 'Batch56', 'Batch54', 'Batch51', 'Batch49', 'Batch52', 'Batch55', 'Batch53', 'Batch50', 'Batch5', 'Batch48', 'Batch47', 'Batch43', 'Batch46', 'Batch40', 'Batch42', 'Batch45', 'Batch4', 'Batch41', 'Batch44', 'Batch39', 'Batch38', 'Batch37', 'Batch36', 'Batch35', 'Batch34', 'Batch29', 'Batch27', 'Batch30', 'Batch3', 'Batch32', 'Batch28', 'Batch33', 'Batch31', 'Batch26', 'Batch25', 'Batch24', 'Batch23', 'Batch22', 'Batch21', 'Batch16', 'Batch19', 'Batch18', 'Batch20', 'Batch15', 'Batch2', 'Batch17', 'Batch14', 'Batch13', 'Batch12', 'Batch11', 'Batch10', 'Batch1']
['Batch9.npy', 'Batch88.npy', 'Batch87.n

In [ ]:
'''
currentPart = part7
batches_list = os.listdir(train_video_path)
print(batches_list)
saved_batches = os.listdir(saved_nparray_path) #no need for this if we have good reources
print(saved_batches)
counter = 0
for batch in batches_list:
        if batch+"Dup6.npy" in saved_batches: #no need for the if we have no resources
          skippedBatch = np.load(os.path.join(train_nparray_path, "data" , batch+".npy"))
          print(batch, "Dup6 skipped", skippedBatch.shape)
          continue
        print(batch)
        x_batch, y_batch = create_Data(os.path.join(train_video_path, batch))
        x_batch = np.asarray(x_batch)
        y_batch = np.asarray(y_batch)
        np.save(os.path.join(train_nparray_path, "data", batch+"Dup6"), x_batch)
        np.save(os.path.join(train_nparray_path, "label", batch+"Dup6"), y_batch)

        counter += 1
        print(counter)
'''

'\ncurrentPart = part7\nbatches_list = os.listdir(train_video_path)\nprint(batches_list)\nsaved_batches = os.listdir(saved_nparray_path) #no need for this if we have good reources\nprint(saved_batches)\ncounter = 0\nfor batch in batches_list:\n        if batch+"Dup6.npy" in saved_batches: #no need for the if we have no resources\n          skippedBatch = np.load(os.path.join(train_nparray_path, "data" , batch+".npy"))\n          print(batch, "Dup6 skipped", skippedBatch.shape)\n          continue    \n        print(batch)\n        x_batch, y_batch = create_Data(os.path.join(train_video_path, batch))\n        x_batch = np.asarray(x_batch)\n        y_batch = np.asarray(y_batch)\n        np.save(os.path.join(train_nparray_path, "data", batch+"Dup6"), x_batch)\n        np.save(os.path.join(train_nparray_path, "label", batch+"Dup6"), y_batch)\n\n        counter += 1\n        print(counter)\n'